In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

In [ ]:
training = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
target = 'label'
trainSplitRandomSeed = 1

In [ ]:
X_training = training.loc[:, training.columns != target]
X_training = X_training / 255.0
test = test / 255.0
y_training = training[target]

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X_training, y_training, stratify = y_training,
                                                                random_state = trainSplitRandomSeed, test_size=0.1)

In [ ]:
def CreateNNModel(inputShape, outputShape, dropoutRate):
    model = Sequential()
    model.add(Dense(inputShape, activation='relu', input_dim=inputShape))
    model.add(Dense(inputShape * 2, activation='relu'))
    model.add(Dense(int(inputShape * 1.5), activation='relu'))
    model.add(Dropout(dropoutRate))
    model.add(Dense(inputShape, activation='relu'))
    model.add(Dense(outputShape, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
nnRegressor = KerasClassifier(build_fn=CreateNNModel, verbose=1)

estimators = [('Scaler', StandardScaler()), ('nn', nnRegressor)]

pipeline = Pipeline(estimators)
param_grid = {
  'nn__inputShape': [784],
  'nn__dropoutRate': [0.2],
  'nn__outputShape': [10],
  'nn__batch_size' : [100],
  'nn__validation_split' : [0.1],
  'nn__epochs' : [30]
}

In [ ]:
reg = GridSearchCV(pipeline, param_grid, cv=3)

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
test.index = np.arange(1, len(test) + 1)
test['Pred_result'] = reg.predict(test)

In [ ]:
test['Pred_result'].to_csv('result.txt', header = ['Label'], index_label = 'ImageId')